# Modelo de Classificação: Regressão Logística para Tipos de Acidente

Este notebook implementa um modelo de **Regressão Logística** para prever o `TipoAcidente` com base na `hora_do_dia`. Conforme discutido, a Regressão Linear não é adequada para prever variáveis categóricas, por isso optamos pela Regressão Logística, que é um modelo de classificação.

In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Carregamento e Preparação dos Dados

Carregamos os dados do arquivo JSON e realizamos as mesmas etapas de pré-processamento do notebook anterior, incluindo a extração da `hora_do_dia`.

In [ ]:
json_path = 'datatran_consolidado.json'

# Simulação da criação do arquivo JSON com os dados fornecidos pelo usuário
# Em um cenário real, este arquivo já existiria.
data_sample = [
    {'data_inversa': '01/01/2020', 'dia_semana': 'quarta-feira', 'horario': '05:40:00', 'uf': 'PA', 'municipio': 'SAO FRANCISCO DO PARA', 'tipo_acidente': 'Saidadeleitocarrosavel', 'condicao_metereologica': 'Ceu Claro'},
    {'data_inversa': '01/01/2020', 'dia_semana': 'quarta-feira', 'horario': '06:00:00', 'uf': 'MG', 'municipio': 'UBERABA', 'tipo_acidente': 'Colisaotransversal', 'condicao_metereologica': 'Ceu Claro'},
    {'data_inversa': '01/01/2020', 'dia_semana': 'quarta-feira', 'horario': '06:00:00', 'uf': 'BA', 'municipio': 'CANUDOS', 'tipo_acidente': 'Saidadeleitocarrosavel', 'condicao_metereologica': 'Nublado'},
    {'data_inversa': '01/01/2020', 'dia_semana': 'quarta-feira', 'horario': '10:08:00', 'uf': 'SP', 'municipio': 'APARECIDA', 'tipo_acidente': 'Colisaotraseira', 'condicao_metereologica': 'Sol'},
    {'data_inversa': '01/01/2020', 'dia_semana': 'quarta-feira', 'horario': '12:10:00', 'uf': 'MG', 'municipio': 'JUATUBA', 'tipo_acidente': 'Saidadeleitocarrosavel', 'condicao_metereologica': 'Ceu Claro'},
    {'data_inversa': '28/07/2025', 'dia_semana': 'segunda-feira', 'horario': '08:50:00', 'uf': 'CE', 'municipio': 'FORTALEZA', 'tipo_acidente': 'Colisaotraseira', 'condicao_metereologica': 'Ceu Claro'},
    {'data_inversa': '26/04/2025', 'dia_semana': 'sábado', 'horario': '13:30:00', 'uf': 'MG', 'municipio': 'ARAGUARI', 'tipo_acidente': 'Saidadeleitocarrosavel', 'condicao_metereologica': 'Nublado'},
    {'data_inversa': '28/08/2025', 'dia_semana': 'quinta-feira', 'horario': '19:20:00', 'uf': 'PR', 'municipio': 'FOZ DO IGUACU', 'tipo_acidente': 'Saidadeleitocarrosavel', 'condicao_metereologica': 'Ceu Claro'},
    {'data_inversa': '11/06/2025', 'dia_semana': 'quarta-feira', 'horario': '10:30:00', 'uf': 'PR', 'municipio': 'SANTO ANTONIO DO SUDOESTE', 'tipo_acidente': 'Colisaotransversal', 'condicao_metereologica': 'Ceu Claro'},
    {'data_inversa': '04/05/2025', 'dia_semana': 'domingo', 'horario': '20:40:00', 'uf': 'PR', 'municipio': 'CURITIBA', 'tipo_acidente': 'AtropelamentodePedestre', 'condicao_metereologica': 'Ceu Claro'}
]

with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(data_sample, f, ensure_ascii=False, indent=4)

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Renomear colunas
df.rename(columns={'data_inversa': 'data', 'dia_semana': 'DiaSemana', 'tipo_acidente': 'TipoAcidente', 'condicao_metereologica': 'CondicaoMetereologica'}, inplace=True)

# Filtrar linhas onde TipoAcidente não é 'none'
df = df[df['TipoAcidente'].astype(str).str.lower() != 'none']

# Remover espaços do TipoAcidente
df['TipoAcidente'] = df['TipoAcidente'].str.replace(' ', '')

# Converter a coluna 'horario' para o formato de hora e extrair apenas a hora
df['hora_do_dia'] = pd.to_datetime(df['horario'], format='%H:%M:%S').dt.hour

df.head()

## 2. Divisão dos Dados e Treinamento do Modelo

Dividimos os dados em conjuntos de treino e teste e treinamos o modelo de Regressão Logística. É importante notar que, com um conjunto de dados muito pequeno e muitas classes de `TipoAcidente`, a capacidade do modelo de aprender padrões significativos será limitada.

In [ ]:
# Variável independente (Feature): hora_do_dia
X = df[['hora_do_dia']]

# Variável dependente (Target): TipoAcidente
y = df['TipoAcidente']

# Dividir os dados em conjuntos de treino e teste
# Usamos 'stratify=y' para garantir que a proporção das classes seja mantida nos conjuntos de treino e teste, se possível.
# A condição 'if len(y.unique()) > 1 else None' é para evitar erro caso haja apenas uma classe, o que pode acontecer com datasets muito pequenos.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y if len(y.unique()) > 1 else None)

# Inicializar e treinar o modelo de Regressão Logística
# 'solver='liblinear'' é uma boa escolha para datasets pequenos.
# 'max_iter=1000' aumenta o número máximo de iterações para garantir a convergência.
model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
model.fit(X_train, y_train)

## 3. Avaliação do Modelo

Avaliamos o desempenho do modelo usando a acurácia e um relatório de classificação. Devido ao tamanho reduzido do conjunto de dados de exemplo, os resultados podem não ser representativos de um cenário real.

In [ ]:
# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do Modelo: {accuracy:.2f}")

# Relatório de Classificação
# Adicionamos um tratamento para o caso de haver poucas classes ou amostras no conjunto de teste,
# o que pode causar erros no classification_report.
report = "Não foi possível gerar o relatório de classificação devido ao número limitado de amostras e classes no conjunto de teste."
if len(y_test.unique()) > 1 and len(y_pred) > 0:
    try:
        report = classification_report(y_test, y_pred, zero_division=0)
    except ValueError as e:
        report = f"Erro ao gerar relatório de classificação: {e}. Isso pode ocorrer com poucas amostras ou classes desbalanceadas."
print("
Relatório de Classificação:
")
print(report)

## 4. Considerações Finais

Com um conjunto de dados maior e mais balanceado, e possivelmente com mais variáveis preditoras (features), o modelo de Regressão Logística teria um desempenho mais robusto e seria capaz de identificar padrões mais complexos na previsão do tipo de acidente com base na hora do dia.